In [1]:
import os
import json
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy import stats
import matplotlib.pyplot as plt

from compute_tempo_aist import *
from aist_pos1s_extraction import *
from aist_pos1s_EsTempo import *
# coco={    
# 0: "nose", 1: "left_eye", 2: "right_eye", 3: "left_ear",4: "right_ear",5: "left_shoulder",
# 6: "right_shoulder",7: "left_elbow",8: "right_elbow",9: "left_wrist",10: "right_wrist",
# 11: "left_hip",12: "right_hip",13: "left_knee",14: "right_knee",15: "left_ankle",16: "right_ankle",}  

def load_pickle(filepath):
    with open(filepath, "rb") as f:
        json_data = pickle.load(f)
    return json_data

def save_to_pickle(filepath, data):
    # filepath = os.path.join(savepath, filename)
    with open(filepath, "wb") as f:
        pickle.dump(data, f)

In [ ]:
marker_dict = {9: "left_wrist", 10: "right_wrist", 
                15: "left_ankle", 16: "right_ankle", 
                }   # 11: "left_hip",12: "right_hip"

config1 = {"sub_dir": ["hand"], "mode": ["zero_uni", "zero_bi"], 
           "markerA_id": [9, 10], "a": 60, "b": 140, "metric": ["pos", "vel"]}
config2 = {"sub_dir": ["foot"], "mode": ["zero_uni", "zero_bi"], 
           "markerA_id": [15, 16], "a": 60, "b": 140, "metric": ["pos", "vel"]}

# config3 = {"sub_dir": ["hand"], "mode": ["zero_uni", "zero_bi"], "markerA_id": [9, 10], "a": 60, "b": 180}
# config4 = {"sub_dir": ["foot"], "mode": ["zero_uni", "zero_bi"], "markerA_id": [15, 16], "a": 60, "b": 180}

configs = [config1, config2]

for cfg in configs:
    a = cfg["a"]
    b = cfg["b"]
    # create_dir(f"tempo_{a}_{b}")
    for sub_dir in cfg["sub_dir"]:
        for mode in cfg["mode"]:
            for markerA_id in cfg["markerA_id"]:
                for metric in cfg["metric"]:
                    
                    savepath = f"./results/one_sensor/tempo_{a}_{b}/{metric}/tempo_data"
                    pickle_filename = f"./results/one_sensor/tempo_{a}_{b}/{metric}/{sub_dir}/{marker_dict[markerA_id]}_{mode}_{a}_{b}.pkl"
                    
                    aist_pos1s(a, b, mode, markerA_id, pickle_filename,savepath, 
                               w_sec = 5,
                               vel_mode= "on" if metric == "vel" else "off")

## March 11 Updates

## Onset Extraction Cell

In [ ]:
marker_dict = {9: "left_wrist", 10: "right_wrist", 
                15: "left_ankle", 16: "right_ankle", 
                }   # 11: "left_hip",12: "right_hip"

config1 = {"sub_dir": ["hand"], "mode": ["zero_uni", "zero_bi"], 
           "markerA_id": [9, 10], "a": 60, "b": 140, "metric": ["pos", "vel"]}
config2 = {"sub_dir": ["foot"], "mode": ["zero_uni", "zero_bi"], 
           "markerA_id": [15, 16], "a": 60, "b": 140, "metric": ["pos", "vel"]}

# config3 = {"sub_dir": ["hand"], "mode": ["zero_uni", "zero_bi"], "markerA_id": [9, 10], "a": 60, "b": 180}
# config4 = {"sub_dir": ["foot"], "mode": ["zero_uni", "zero_bi"], "markerA_id": [15, 16], "a": 60, "b": 180}

configs = [config1, config2]

for cfg in configs:
    a = cfg["a"]
    b = cfg["b"]
    # create_dir(f"tempo_{a}_{b}")
    for sub_dir in cfg["sub_dir"]:
        for mode in cfg["mode"]:
            for markerA_id in cfg["markerA_id"]:
                for metric in cfg["metric"]:
                    
                    savepath = f"./extracted_body_onsets/{metric}"           
                    extract_body_onsets(mode, markerA_id,savepath, 
                               vel_mode= "on" if metric == "vel" else "off")

## Estimate Tempo

In [3]:
json_filename = "music_id_tempo.json"
with open(json_filename, "r") as file:
    aist_tempo = json.load(file)
    
def create_dir(main_dir, tempo_dir):
    # main_dir = "/itf-fi-ml/home/sagardu/aist_tempo_est/saved_result"
    directories = [f"{tempo_dir}/pos", f"{tempo_dir}/vel",
                   f"{tempo_dir}/tempo_data/pos", f"{tempo_dir}/tempo_data/vel",]
    
    for dir_path in directories:
        full_path = os.path.join(main_dir, dir_path)
        os.makedirs(full_path, exist_ok=True)
        
segment_keys = ["both_hand_x", "both_hand_y", "both_foot_x", "both_foot_y", 
                "lefthand_xy", "righthand_xy", "leftfoot_xy", "rightfoot_xy", 
                "left_hand_x", "right_hand_x", "left_hand_y", "right_hand_y", 
                "left_foot_x", "right_foot_x", "left_foot_y", "right_foot_y", 
                "both_hand_resultant", "both_foot_resultant", "left_hand_resultant", 
                "right_hand_resultant", "left_foot_resultant", "right_foot_resultant"]
result = { key: {
    "filename": [],
    "dance_genre": [],
    "situation": [],
    "camera_id": [],
    "dancer_id": [],
    "music_id": [],
    "choreo_id": [],
    "music_tempo": [],
    "estimated_bpm_per_window": [],
    "magnitude_per_window": [],
    "bpm_avg": [],
    "bpm_mode": [],
    "bpm_median": [],
} for key in segment_keys }

tempo_data = {}

fps = 60
w_sec = 5
h_sec = w_sec/2
window_size = int(fps*w_sec)
hop_size = int(fps*h_sec)

a = 60; b =140
tempi_range = np.arange(a,b,1)
metric = "vel"
mode = "zero_uni"

main_dir = "/itf-fi-ml/home/sagardu/aist_tempo_est/saved_result"
create_dir(main_dir, f"tempo_{a}_{b}")

save_dir = f"./saved_result/tempo_{a}_{b}/"
onset_dir = f"./extracted_body_onsets/{metric}/"
f_path = "./aist_dataset/aist_annotation/keypoints2d"
aist_filelist = os.listdir(f_path)


count= 0
for idx, filename in enumerate(tqdm(aist_filelist)):
    
    file_info = filename.split("_")
    dance_genre = file_info[0] 
    situation = file_info[1] 
    camera_id = file_info[2] 
    dancer_id = file_info[3]
    music_id = file_info[4]
    choreo_id = file_info[5].strip(".pkl")
    
    test_path = os.path.join(onset_dir, "ax0", f"left_wrist_{mode}_{filename}")
    isExist = os.path.exists(test_path) 
    if not isExist:
        continue
                            
    left_hand_x  = load_pickle(os.path.join(onset_dir, "ax0", f"left_wrist_{mode}_{filename}"))
    left_hand_y  = load_pickle(os.path.join(onset_dir, "ax1", f"left_wrist_{mode}_{filename}"))
    
    right_hand_x = load_pickle(os.path.join(onset_dir, "ax0", f"right_wrist_{mode}_{filename}"))
    right_hand_y = load_pickle(os.path.join(onset_dir, "ax1", f"right_wrist_{mode}_{filename}"))
    
    left_foot_x  = load_pickle(os.path.join(onset_dir, "ax0", f"left_ankle_{mode}_{filename}"))
    left_foot_y  = load_pickle(os.path.join(onset_dir, "ax1", f"left_ankle_{mode}_{filename}"))
    
    right_foot_x = load_pickle(os.path.join(onset_dir, "ax0", f"right_ankle_{mode}_{filename}"))
    right_foot_y = load_pickle(os.path.join(onset_dir, "ax1", f"right_ankle_{mode}_{filename}"))
    
    novelty_length = left_hand_x['raw_signal'].shape[0]
    
    key = 'sensor_onsets'
    thres = 0.2
    
    bothhand_x = filter_dir_onsets_by_threshold((left_hand_x[key] + right_hand_x[key]), threshold_s= thres, fps=fps)
    bothhand_y = filter_dir_onsets_by_threshold((left_hand_y[key] + right_hand_y[key]), threshold_s= thres, fps=fps)

    bothfoot_x = filter_dir_onsets_by_threshold((left_foot_x[key] + right_foot_x[key]), threshold_s= thres, fps=fps)
    bothfoot_y = filter_dir_onsets_by_threshold((left_foot_y[key] + right_foot_y[key]), threshold_s= thres, fps=fps)
    
    lefthand_xy = filter_dir_onsets_by_threshold((left_hand_x[key] + left_hand_y[key]), threshold_s= thres, fps=fps)
    righthand_xy = filter_dir_onsets_by_threshold((right_hand_x[key] + right_hand_y[key]), threshold_s= thres, fps=fps)

    leftfoot_xy = filter_dir_onsets_by_threshold((left_foot_x[key] + left_foot_y[key]), threshold_s= thres, fps=fps)
    rightfoot_xy = filter_dir_onsets_by_threshold((right_foot_x[key] + right_foot_y[key]), threshold_s= thres, fps=fps)
    
    # Resultant part
    key1 = 'resultant_onsets'
    left_hand_resultant  = load_pickle(os.path.join(onset_dir, "resultant", f"left_wrist_{mode}_{filename}"))
    right_hand_resultant  = load_pickle(os.path.join(onset_dir, "resultant", f"right_wrist_{mode}_{filename}"))

    left_foot_resultant = load_pickle(os.path.join(onset_dir, "resultant", f"left_ankle_{mode}_{filename}"))
    right_foot_resultant = load_pickle(os.path.join(onset_dir, "resultant", f"right_ankle_{mode}_{filename}"))
    
    both_hand_resultant = filter_dir_onsets_by_threshold((left_hand_resultant[key1] + right_hand_resultant[key1]), threshold_s= thres, fps=fps)
    both_foot_resultant = filter_dir_onsets_by_threshold((left_foot_resultant[key1] + right_foot_resultant[key1]), threshold_s= thres, fps=fps)
    
    segment_ax = {
                "both_hand_x": bothhand_x, "both_hand_y": bothhand_y, "both_foot_x": bothfoot_x, "both_foot_y": bothfoot_y,
                "lefthand_xy": lefthand_xy, "righthand_xy": righthand_xy, "leftfoot_xy": leftfoot_xy, "rightfoot_xy": rightfoot_xy,
                
                "left_hand_x": left_hand_x[key], "right_hand_x": right_hand_x[key], 
                "left_hand_y": left_hand_y[key], "right_hand_y": right_hand_y[key],
                
                "left_foot_x": left_foot_x[key], "right_foot_x": right_foot_x[key],
                "left_foot_y": left_foot_y[key], "right_foot_y": right_foot_y[key],
                
                "both_hand_resultant": both_hand_resultant, "both_foot_resultant": both_foot_resultant,                         
                "left_hand_resultant": left_hand_resultant[key1], "right_hand_resultant": right_hand_resultant[key1],
                "left_foot_resultant": left_foot_resultant[key1], "right_foot_resultant": right_foot_resultant[key1],
                }
    
    for seg_key, seg in segment_ax.items():
        
        sensor_onsets = binary_to_peak(seg, peak_duration=0.05)
        
        tempogram_ab, tempogram_raw, time_axis_seconds, tempo_axis_bpm = compute_tempogram(sensor_onsets, fps, 
                                                                        window_length=window_size, hop_size=hop_size, tempi=tempi_range)
        

        tempo_data_maxmethod = dance_beat_tempo_estimation_maxmethod(tempogram_ab, tempogram_raw, fps, 
                                                        novelty_length, window_size, hop_size, tempi_range)
    
        tempo_data[seg_key] = tempo_data_maxmethod
        
        estimated_bpm_per_window = tempo_data_maxmethod["bpm_arr"]
        magnitude_per_window = tempo_data_maxmethod["mag_arr"]
        
        tempo_avg = np.round(np.average(estimated_bpm_per_window), 2)     # mean
        tempo_mode = stats.mode(estimated_bpm_per_window.flatten())[0]        # 
        tempo_median = np.median(estimated_bpm_per_window.flatten())

        # Append results for the current segment
        result[seg_key]["filename"].append(filename.strip(".pkl"))
        result[seg_key]["dance_genre"].append(dance_genre)
        result[seg_key]["situation"].append(situation)
        result[seg_key]["camera_id"].append(camera_id)
        result[seg_key]["dancer_id"].append(dancer_id)
        result[seg_key]["music_id"].append(music_id)
        result[seg_key]["choreo_id"].append(choreo_id)
        result[seg_key]["music_tempo"].append(aist_tempo[music_id])
        result[seg_key]["estimated_bpm_per_window"].append(estimated_bpm_per_window)
        result[seg_key]["magnitude_per_window"].append(magnitude_per_window)
        result[seg_key]["bpm_avg"].append(tempo_avg)
        result[seg_key]["bpm_mode"].append(tempo_mode)
        result[seg_key]["bpm_median"].append(tempo_median)
    count +=1
print("total processed:",count)    
for seg_key in segment_keys:
    
    fname1 = f"{metric}/{seg_key}_{mode}_W{w_sec}_H{h_sec}_{a}_{b}.pkl"
    fpath1 = os.path.join(save_dir, fname1)
    df_seg = pd.DataFrame(result[seg_key])
    df_seg.to_pickle(fpath1)
    
    tempodata_fname = f"tempo_data/{metric}/{seg_key}_{mode}_W{w_sec}_H{h_sec}_{a}_{b}_tempo_data.pkl"
    fpath2 = os.path.join(save_dir, tempodata_fname)
    save_to_pickle(fpath2, tempo_data[seg_key])
    
    print(f"Saved {fname1}")

100%|██████████| 1510/1510 [04:16<00:00,  5.89it/s]


total processed: 1341
Saved vel/both_hand_x_zero_uni_W5_H2.5_60_140.pkl
Saved vel/both_hand_y_zero_uni_W5_H2.5_60_140.pkl
Saved vel/both_foot_x_zero_uni_W5_H2.5_60_140.pkl
Saved vel/both_foot_y_zero_uni_W5_H2.5_60_140.pkl
Saved vel/lefthand_xy_zero_uni_W5_H2.5_60_140.pkl
Saved vel/righthand_xy_zero_uni_W5_H2.5_60_140.pkl
Saved vel/leftfoot_xy_zero_uni_W5_H2.5_60_140.pkl
Saved vel/rightfoot_xy_zero_uni_W5_H2.5_60_140.pkl
Saved vel/left_hand_x_zero_uni_W5_H2.5_60_140.pkl
Saved vel/right_hand_x_zero_uni_W5_H2.5_60_140.pkl
Saved vel/left_hand_y_zero_uni_W5_H2.5_60_140.pkl
Saved vel/right_hand_y_zero_uni_W5_H2.5_60_140.pkl
Saved vel/left_foot_x_zero_uni_W5_H2.5_60_140.pkl
Saved vel/right_foot_x_zero_uni_W5_H2.5_60_140.pkl
Saved vel/left_foot_y_zero_uni_W5_H2.5_60_140.pkl
Saved vel/right_foot_y_zero_uni_W5_H2.5_60_140.pkl
Saved vel/both_hand_resultant_zero_uni_W5_H2.5_60_140.pkl
Saved vel/both_foot_resultant_zero_uni_W5_H2.5_60_140.pkl
Saved vel/left_hand_resultant_zero_uni_W5_H2.5_60_140.pk

#### Skipped Files Analysis

In [ ]:
Skipped_files = ['gPO_sMM_cAll_d12_mPO2_ch07.pkl', 'gBR_sBM_cAll_d04_mBR2_ch05.pkl', 'gJS_sFM_cAll_d03_mJS1_ch02.pkl', 'gMH_sMM_cAll_d24_mMH3_ch10.pkl', 'gLH_sMM_cAll_d18_mLH5_ch10.pkl', 'gJS_sMM_cAll_d02_mJS3_ch03.pkl', 'gBR_sBM_cAll_d05_mBR1_ch10.pkl', 'gPO_sMM_cAll_d11_mPO2_ch04.pkl', 'gBR_sFM_cAll_d06_mBR5_ch21.pkl', 'gJB_sMM_cAll_d09_mJB5_ch10.pkl', 'gLO_sMM_cAll_d15_mLO4_ch09.pkl', 'gBR_sMM_cAll_d06_mBR1_ch07.pkl', 'gHO_sMM_cAll_d19_mHO2_ch02.pkl', 'gBR_sBM_cAll_d04_mBR3_ch08.pkl', 'gLO_sMM_cAll_d14_mLO5_ch06.pkl', 'gJS_sFM_cAll_d03_mJS5_ch13.pkl', 'gJS_sMM_cAll_d01_mJS3_ch03.pkl', 'gLH_sMM_cAll_d17_mLH3_ch05.pkl', 'gWA_sMM_cAll_d27_mWA0_ch07.pkl', 'gKR_sMM_cAll_d30_mKR1_ch08.pkl', 'gHO_sFM_cAll_d20_mHO1_ch09.pkl', 'gBR_sBM_cAll_d05_mBR4_ch07.pkl', 'gHO_sMM_cAll_d21_mHO5_ch10.pkl', 'gBR_sBM_cAll_d04_mBR1_ch07.pkl', 'gWA_sMM_cAll_d25_mWA4_ch02.pkl', 'gLH_sMM_cAll_d17_mLH5_ch06.pkl', 'gJS_sFM_cAll_d02_mJS1_ch02.pkl', 'gPO_sMM_cAll_d11_mPO3_ch05.pkl', 'gBR_sMM_cAll_d05_mBR2_ch05.pkl', 'gMH_sMM_cAll_d23_mMH2_ch06.pkl', 'gBR_sBM_cAll_d05_mBR1_ch08.pkl', 'gBR_sBM_cAll_d05_mBR1_ch07.pkl', 'gJB_sMM_cAll_d07_mJB4_ch02.pkl', 'gBR_sBM_cAll_d06_mBR3_ch07.pkl', 'gBR_sMM_cAll_d06_mBR3_ch08.pkl', 'gBR_sBM_cAll_d05_mBR0_ch10.pkl', 'gBR_sMM_cAll_d06_mBR4_ch09.pkl', 'gJB_sMM_cAll_d07_mJB5_ch03.pkl', 'gJS_sFM_cAll_d02_mJS1_ch11.pkl', 'gLH_sMM_cAll_d16_mLH0_ch01.pkl', 'gLO_sMM_cAll_d13_mLO1_ch02.pkl', 'gJS_sFM_cAll_d01_mJS1_ch02.pkl', 'gMH_sMM_cAll_d24_mMH1_ch08.pkl', 'gBR_sBM_cAll_d04_mBR2_ch10.pkl', 'gPO_sMM_cAll_d10_mPO3_ch02.pkl', 'gKR_sFM_cAll_d30_mKR0_ch15.pkl', 'gMH_sMM_cAll_d22_mMH1_ch02.pkl', 'gBR_sMM_cAll_d05_mBR1_ch04.pkl', 'gMH_sMM_cAll_d23_mMH1_ch05.pkl', 'gBR_sBM_cAll_d05_mBR0_ch07.pkl', 'gKR_sFM_cAll_d29_mKR5_ch13.pkl', 'gJB_sMM_cAll_d07_mJB3_ch01.pkl', 'gBR_sBM_cAll_d04_mBR0_ch10.pkl', 'gJS_sFM_cAll_d02_mJS5_ch10.pkl', 'gLO_sMM_cAll_d14_mLO4_ch05.pkl', 'gJS_sFM_cAll_d03_mJS4_ch12.pkl', 'gLH_sMM_cAll_d17_mLH0_ch04.pkl', 'gLH_sMM_cAll_d16_mLH3_ch03.pkl', 'gKR_sMM_cAll_d29_mKR0_ch04.pkl', 'gJS_sFM_cAll_d01_mJS0_ch01.pkl', 'gHO_sMM_cAll_d20_mHO2_ch04.pkl', 'gLO_sMM_cAll_d13_mLO5_ch03.pkl', 'gBR_sFM_cAll_d04_mBR5_ch06.pkl', 'gKR_sMM_cAll_d30_mKR2_ch09.pkl', 'gKR_sMM_cAll_d28_mKR2_ch03.pkl', 'gHO_sMM_cAll_d20_mHO4_ch06.pkl', 'gBR_sMM_cAll_d04_mBR4_ch02.pkl', 'gBR_sBM_cAll_d05_mBR1_ch09.pkl', 'gBR_sFM_cAll_d04_mBR4_ch05.pkl', 'gHO_sMM_cAll_d19_mHO3_ch03.pkl', 'gBR_sBM_cAll_d04_mBR3_ch05.pkl', 'gBR_sBM_cAll_d06_mBR5_ch08.pkl', 'gBR_sMM_cAll_d04_mBR5_ch03.pkl', 'gBR_sFM_cAll_d06_mBR2_ch16.pkl', 'gBR_sBM_cAll_d04_mBR3_ch07.pkl', 'gLH_sMM_cAll_d18_mLH1_ch07.pkl', 'gBR_sBM_cAll_d06_mBR5_ch05.pkl', 'gHO_sFM_cAll_d19_mHO3_ch04.pkl', 'gBR_sMM_cAll_d06_mBR5_ch10.pkl', 'gJS_sMM_cAll_d02_mJS0_ch04.pkl', 'gMH_sMM_cAll_d24_mMH2_ch09.pkl', 'gHO_sMM_cAll_d20_mHO3_ch05.pkl', 'gJS_sMM_cAll_d02_mJS1_ch02.pkl', 'gPO_sMM_cAll_d12_mPO3_ch08.pkl', 'gJS_sMM_cAll_d03_mJS5_ch05.pkl', 'gBR_sBM_cAll_d06_mBR2_ch10.pkl', 'gBR_sMM_cAll_d04_mBR3_ch01.pkl', 'gJB_sFM_cAll_d08_mJB0_ch08.pkl', 'gPO_sMM_cAll_d12_mPO5_ch09.pkl', 'gJB_sMM_cAll_d09_mJB1_ch08.pkl', 'gBR_sBM_cAll_d06_mBR3_ch08.pkl', 'gMH_sFM_cAll_d22_mMH1_ch02.pkl', 'gHO_sFM_cAll_d20_mHO0_ch08.pkl', 'gBR_sBM_cAll_d05_mBR0_ch08.pkl', 'gWA_sMM_cAll_d25_mWA5_ch03.pkl', 'gBR_sFM_cAll_d04_mBR1_ch02.pkl', 'gKR_sMM_cAll_d29_mKR5_ch06.pkl', 'gJS_sMM_cAll_d03_mJS0_ch01.pkl', 'gWA_sMM_cAll_d26_mWA5_ch06.pkl', 'gJB_sFM_cAll_d07_mJB3_ch07.pkl', 'gLO_sMM_cAll_d15_mLO5_ch10.pkl', 'gLH_sMM_cAll_d18_mLH3_ch09.pkl', 'gWA_sMM_cAll_d26_mWA3_ch05.pkl', 'gBR_sFM_cAll_d04_mBR4_ch07.pkl', 'gJB_sMM_cAll_d09_mJB0_ch07.pkl', 'gBR_sBM_cAll_d05_mBR5_ch08.pkl', 'gLO_sMM_cAll_d13_mLO0_ch01.pkl', 'gLO_sMM_cAll_d15_mLO0_ch07.pkl', 'gBR_sBM_cAll_d04_mBR0_ch08.pkl', 'gPO_sMM_cAll_d10_mPO5_ch03.pkl', 'gMH_sMM_cAll_d23_mMH0_ch04.pkl', 'gJS_sMM_cAll_d03_mJS1_ch02.pkl', 'gBR_sBM_cAll_d05_mBR5_ch10.pkl', 'gBR_sBM_cAll_d04_mBR3_ch10.pkl', 'gBR_sFM_cAll_d05_mBR4_ch11.pkl', 'gLH_sMM_cAll_d18_mLH2_ch08.pkl', 'gJS_sFM_cAll_d01_mJS3_ch04.pkl', 'gWA_sMM_cAll_d27_mWA2_ch08.pkl', 'gBR_sBM_cAll_d06_mBR4_ch08.pkl', 'gJS_sFM_cAll_d01_mJS1_ch07.pkl', 'gJB_sFM_cAll_d07_mJB2_ch03.pkl', 'gKR_sMM_cAll_d28_mKR1_ch02.pkl', 'gBR_sBM_cAll_d04_mBR1_ch10.pkl', 'gHO_sFM_cAll_d20_mHO5_ch13.pkl', 'gLO_sMM_cAll_d14_mLO1_ch04.pkl', 'gJB_sFM_cAll_d09_mJB1_ch16.pkl', 'gMH_sMM_cAll_d22_mMH2_ch03.pkl', 'gBR_sFM_cAll_d04_mBR3_ch04.pkl', 'gBR_sFM_cAll_d05_mBR5_ch14.pkl', 'gPO_sMM_cAll_d12_mPO4_ch10.pkl', 'gHO_sMM_cAll_d19_mHO0_ch01.pkl', 'gWA_sMM_cAll_d25_mWA1_ch01.pkl', 'gKR_sFM_cAll_d28_mKR5_ch06.pkl', 'gKR_sMM_cAll_d29_mKR1_ch05.pkl', 'gKR_sFM_cAll_d28_mKR3_ch07.pkl', 'gKR_sMM_cAll_d28_mKR0_ch01.pkl', 'gMH_sMM_cAll_d22_mMH0_ch01.pkl', 'gHO_sMM_cAll_d21_mHO2_ch07.pkl', 'gPO_sMM_cAll_d11_mPO5_ch06.pkl', 'gJS_sFM_cAll_d02_mJS3_ch04.pkl', 'gBR_sBM_cAll_d05_mBR4_ch10.pkl', 'gBR_sMM_cAll_d05_mBR4_ch06.pkl', 'gHO_sMM_cAll_d21_mHO4_ch09.pkl', 'gHO_sFM_cAll_d20_mHO3_ch14.pkl', 'gKR_sMM_cAll_d30_mKR0_ch07.pkl', 'gWA_sMM_cAll_d27_mWA5_ch10.pkl', 'gJS_sMM_cAll_d01_mJS1_ch02.pkl', 'gBR_sBM_cAll_d04_mBR2_ch08.pkl', 'gJS_sFM_cAll_d02_mJS0_ch08.pkl', 'gJS_sMM_cAll_d01_mJS0_ch01.pkl', 'gWA_sMM_cAll_d27_mWA3_ch09.pkl', 'gLH_sMM_cAll_d16_mLH2_ch02.pkl', 'gPO_sMM_cAll_d10_mPO2_ch01.pkl', 'gJB_sFM_cAll_d09_mJB2_ch17.pkl', 'gBR_sBM_cAll_d04_mBR1_ch08.pkl', 'gLO_sMM_cAll_d15_mLO1_ch08.pkl', 'gJS_sMM_cAll_d03_mJS3_ch03.pkl', 'gWA_sMM_cAll_d26_mWA2_ch04.pkl', 'gBR_sFM_cAll_d04_mBR2_ch03.pkl', 'gBR_sBM_cAll_d05_mBR5_ch07.pkl', 'gJB_sMM_cAll_d08_mJB3_ch05.pkl', 'gJB_sMM_cAll_d09_mJB3_ch09.pkl', 'gHO_sMM_cAll_d21_mHO3_ch08.pkl', 'gBR_sBM_cAll_d05_mBR4_ch08.pkl', 'gMH_sMM_cAll_d24_mMH0_ch07.pkl', 'gBR_sFM_cAll_d05_mBR2_ch09.pkl', 'gJB_sMM_cAll_d08_mJB1_ch04.pkl', 'gJB_sMM_cAll_d08_mJB4_ch06.pkl', 'gKR_sMM_cAll_d30_mKR3_ch10.pkl']

f_path = "./aist_dataset/aist_annotation/keypoints2d"	

for idx, filename in enumerate(tqdm(Skipped_files)):
        
    file_path = os.path.join(f_path, filename)
    with open(file_path, 'rb') as file:
            motion_data = pickle.load(file)

    markerA_x = motion_data["keypoints2d"][0, :, markerA_id, 0]     # array (n,)
    markerA_y = motion_data["keypoints2d"][0, :, markerA_id, 1]      # array (n,)
    
    print(len(markerA_x))
    break

